In [1]:
import numpy as np

In [2]:
# Define Cost function, lambda function, p function, alpha function
def cost(theta:float) -> float:
    return theta
def la(theta:float) -> float:
    return 1/theta
def p(theta:float) -> float:
    return 1/theta
def al(theta:float)-> float:
    return theta
def L_al_la(la,al):
    l=la-al
    return l
def update_lambda(la,al,e):
    beta_1 = 0.023
    la-= e*L_al_la(la,al)-beta_1*1
    return la
def update_alpha(la,al,e):
    beta_2 = 10**8
    al-= e*L_al_la(la,al)-beta_2*1
    return al
# @ param N: is the number of total population
# @ param total_time: the time that we want to simulate the process (6 month)
# @ param x_0: initial infected patients (greater than 0)
def simulation_infected(la, p, al, N, total_time, x_0):
    # initialize time and infected patient
    t_0 = 0
    # bind variables to initial conditions
    x_n = x_0
    assert(x_n!=0)
    t_n = t_0
    # initiate X_n process and corresponding T_n process
    X_n = [x_0]
    T_n = [t_0]
    # converge
    conv =0
    # loop
    # break condition
    # @ condition1: when the time exceeds the total time
    # @ condition2: when the infected patients go to zero
    # @ condition3: when the whole populations are infected! :(
    while (t_n<total_time and
           x_n != 0 and
           x_n < N
          ):
        i = x_n
        # q i _ i+1
        q_forward_i = la*p*2*x_n*(N-i)/(N*(N-1))
        # q i _ i-1
        q_backward_i = al*i
        # waiting time rate v_i = (q i _ i+1) + (q i _ i-1)
        v_i = q_forward_i + q_backward_i
        t_i = np.random.exponential(v_i)
        # jumping probability to STATE i+1 is (q i _ i+1)/v_i
        jump = np.random.binomial(n=1,p=(q_forward_i/v_i))
        if (jump ==1):
            x_n += 1
        else:
            x_n -= 1
        # increase time    
        t_n = t_n+t_i
        # append Process
        X_n.append(x_n)
        T_n.append(t_n)

        if abs(x_n - X_n[-1]) < 10:
            print('converge to C')
            conv = 1
            break
        
    return (X_n,T_n,conv)
e=0.01
para_alpha = 10**(-8)
para_lamda = 0.01
while True:
    _,_,conv = simulation_infected(para_lamda,0.2,para_alpha,20000,100,10)
    if conv ==1:
        break
    para_alpha = update_alpha(para_alpha,e)
    _,_,conv = simulation_infected(para_lamda,0.2,para_alpha,20000,100,10)
    if conv ==1:
        break
    para_lamda = update_lambda(para_lamda,e)
    _,_,conv = simulation_infected(para_lamda,0.2,para_alpha,20000,100,10)
    if conv ==1:
        break

converge to C
